In [ ]:
# set up and test simple commands with LLM 
# swaped OpenAI for Mistral due to expired free tier quota

In [2]:
# import mistral 
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [3]:
import os

In [4]:
# get API key stored in env variable
api_key = os.getenv("MISTRAL_KEY")

In [5]:
client = MistralClient(api_key=api_key)

In [16]:
# ask it a question
response = client.chat(
    model = 'mistral-tiny',
    messages = [ChatMessage(role = "user", content = "Is it too late to join the course?")]
)

In [17]:
print(response.choices[0].message.content)

I'm sorry for any confusion, but I'll need a bit more information to accurately answer your question. The phrase "the course" is quite general. Are you asking about a specific online course, a university course, or some other type of educational program? Many online courses have flexible start dates, while others may have strict enrollment periods. For in-person classes, it's usually important to check well in advance due to limited seating and registration processes. Please provide more details, and I'll do my best to help you.


In [7]:
# set up a search using minisearch from Alexei Grigorev
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-30 19:35:37--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-30 19:35:37 (8.97 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



In [6]:
import minsearch

In [7]:
import json

In [10]:
# get the document
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-30 19:35:43--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.1’

documents.json.1    100%[===================>] 642.90K  --.-KB/s    in 0.02s   

2024-06-30 19:35:43 (26.7 MB/s) - ‘documents.json.1’ saved [658332/658332]



In [8]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [9]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [10]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
)

In [11]:
# index (fit the index to) the documents
index.fit(documents)

In [12]:
# some query (question) 
query = 'the course has already started, can I still enroll?'

In [13]:
# let's search but make some fields more important and some less so
boost = {'question': 3.0, 'section': 0.5}
results = index.search(
    query, 
    filter_dict = {'course': 'data-engineering-zoomcamp'}, 
    boost_dict = boost, 
    # filter_dict = {'course': 'data-engineering-zoomcamp'}, # if we want to search only for specific course 
    num_results = 5
)

In [14]:
print(results)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}, {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', 'section': 'General course-related questions', 'question': 'Course - Can I follow the course after it finishes?', 'course': 'data-engineering-zoomcamp'}, {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. Th

In [18]:
# creating the prompt template
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}
""".strip()

In [19]:
# defining the context
context = ""
for doc in results:
    context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [21]:
prompt = prompt_template.format(question = query, context = context).strip()

In [24]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started, can I still enroll?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fi

In [26]:
# feed it to LLM
response = client.chat(
    model = "mistral-large-latest",
    messages = [ChatMessage(role = "user", content = prompt)]
)
print(response.choices[0].message.content)

Yes, you can still enroll in the course even if it has already started. You're eligible to submit homeworks, but be aware of the deadlines for turning in the final projects.


In [ ]:
# TODO: clean RAG flow, Elasticsearch, HW